# Data-Driven Decks



## Inkscape Driver

> TBD

## Drawio Driver

## Before we begin

> ## This is **NOT READY** for prime-time
> Start the **demo** export server. It will try to install its dependencies with `jlpm`, and requires `nodejs`.
> ```bash
> !python scripts/drawio_export_demo.py
> ```

In [16]:
import base64, html, urllib.parse, pathlib as P, tempfile, asyncio, IPython
import IPython.display as D, networkx as nx, jinja2, babel.support as B, pandas as pd, PyPDF2, traitlets as T, ipywidgets as W, tornado.ioloop
from nbconvert.filters.markdown_mistune import markdown2html_mistune
import httpx

## Slides

In [2]:
class Slides(W.HTML):
    """ some number of slides, as PDF
    """
    pdf = T.Unicode()
    
    def __init__(self, *args, **kwargs):
        if not kwargs.get("layout"):
            kwargs["layout"] = dict(
                display = "flex",
                flex_flow = "column wrap",
                flex = "1",
                height = "100%",
            )
        super().__init__(*args, **kwargs)
    
    @T.observe("pdf")
    def _on_pdf(self, change):
        self.value = f"""
            <iframe 
                src="data:application/pdf;base64,{change.new}" 
                style="border: 0; min-width: 400px; min-height: 400px; width: 100%; height: 100%;">
            </iframe>
        """

### DrawioSlides

In [3]:
class DrawioSlides(Slides):
    """ Slides built with drawio-export from drawio XML
    """
    xml = T.Unicode()
    params = T.Dict()
    url = T.Unicode(default_value="http://localhost:8000")
    
    CORE_PARAMS = dict(
        format="pdf",
        base64="1"
    )
    
    async def update_pdf(self):
        # this really needs to be a queue
        async with httpx.AsyncClient() as client:
            xml = self.transform(self.xml)
            r = await client.post(
                self.url, 
                timeout=None,
                data=dict(
                    xml=xml, 
                    **self.params, 
                    **self.CORE_PARAMS
                ))
            self.pdf = r.text
    
    def transform(self, xml):
        return xml

    @T.observe("xml")
    def _on_xml(self, change=None):
        tornado.ioloop.IOLoop.current().add_callback(self.update_pdf)

In [4]:
HOW_IT_WORKS = P.Path("testfiles/How it works.dio")
how_it_works = DrawioSlides(xml=HOW_IT_WORKS.read_text())
how_it_works

DrawioSlides(value='', layout=Layout(display='flex', flex='1', flex_flow='column wrap', height='100%'))

#### TemplatedDrawioSlides

In [5]:
class TemplatedDrawioSlides(DrawioSlides):
    context = T.Dict()
    
    AMP = "&"
    _AMP_ = "_____AMP_____"

    def clean(self, txt):
        return txt.replace(self.AMP, self._AMP_)

    def smudge(self, txt):
        return txt.replace(self._AMP_, self.AMP)
    
    def transform(self, xml):
        env = jinja2.Environment(
          extensions=['jinja2.ext.i18n', 'jinja2.ext.autoescape'],
          autoescape=jinja2.select_autoescape(['html', 'xml'])
        )

        return self.smudge(env.from_string(self.clean(xml)).render(**{
            key: self.markdown(value)
            for key, value in (self.context or {}).items()
        }))
    
    @T.observe("context")
    def _on_context(self, change):
        self._on_xml()
    
    def markdown(self, md):
        return markdown2html_mistune(md).replace(self.AMP, self._AMP_)

In [6]:
TEMPLATE = P.Path("testfiles/template deck.dio") 
title = TemplatedDrawioSlides(xml=TEMPLATE.read_text())
title

TemplatedDrawioSlides(value='', layout=Layout(display='flex', flex='1', flex_flow='column wrap', height='100%'…

In [7]:
title.context = {"title": "This is the  _TITLE_"}

In [24]:
ideas = []
for i in range(4):
    await asyncio.sleep(2)
    ideas += [
        TemplatedDrawioSlides(xml=TEMPLATE.read_text(), context=dict(
            title=f"Idea {i + 1}",
            abstract=f"This is idea {i + 1}",
            hero=f"# {i + 1}"
        ))
    ]


## Deck

### Deck MK1 Prototype

The simplest deck prototype is just a box.

In [25]:
deck = W.HBox([title, how_it_works, *ideas], 
              layout=dict(display="flex", flex_flow="row wrap"))
# deck

### Deck MK2 Prototype

This builds a composite deck.

In [30]:
class Deck(W.HBox):
    composite = T.Unicode()
    preview = T.Instance(Slides)
    
    def __init__(self, *args, **kwargs):
        if "layout" not in kwargs:
            kwargs["layout"] = dict(display="flex", flex_flow="row wrap")
        super().__init__(*args, **kwargs)
    
    @T.default("preview")
    def _default_preview(self):
        slides = Slides()
        T.dlink((self, "composite"), (slides, "pdf"))
        self.update_composite()
        return slides
    
    def update_composite(self):
        with tempfile.TemporaryDirectory() as td:
            tdp = P.Path(td)
            merger = PyPDF2.PdfFileMerger()
            for i, child in enumerate(self.children):
                next_pdf = (tdp / f"doc-{i}.pdf")
                wrote = next_pdf.write_bytes(base64.b64decode(child.pdf.encode("utf-8")))
                if wrote:
                    merger.append(PyPDF2.PdfFileReader(str(next_pdf)))
            output_pdf = tdp / "output.pdf"
            merger.write(str(output_pdf))
            self.composite = base64.b64encode(output_pdf.read_bytes()).decode("utf-8")

In [31]:
new_deck = Deck(deck.children)

In [32]:
new_deck.preview

Slides(value='\n            <iframe \n                src="data:application/pdf;base64,JVBERi0xLjMKMSAwIG9iago…

In [29]:
new_deck.update_composite()

In [14]:
new_deck

Deck(children=(TemplatedDrawioSlides(value='\n            <iframe \n                src="data:application/pdf;…